In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1673931548347_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
Q400_128WPS_1 = spark.sql('''
                                SELECT

                                ECN,

                                UTC_TIMEDATE,

                                FileName,

                                CAST((MAX(`Time-In-Secs`) - MIN(`Time-In-Secs`)) / 60 AS INT) AS FlightTime

                                FROM precogdb.q400128wps

                                WHERE FlightDate > '2022-10-31'

                                AND `Radio-Altitude` NOT LIKE '%*%'

                                AND `Radio-Altitude` <> ''

                                AND `Main-WOW` = 'Air (0)'

                                --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)

                                GROUP BY ECN, UTC_TIMEDATE, Filename
                            ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
Q400_128WPS_1.createOrReplaceTempView('Q400_128WPS_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
Q400_128WPS_2 = spark.sql('''

                SELECT MIN(`Time-Per-Sec`) AS AirborneTime,
                
                flightdate,
                
                Filename
                
                FROM precogdb.q400128wps
                
                WHERE FlightDate > '2022-10-31'
                        
                AND `Radio-Altitude` NOT LIKE '%*%'
                        
                AND `Radio-Altitude` <> ''
                        
                AND `Main-WOW` = 'Air (0)'

                --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)

                GROUP BY flightdate,Filename
''')

Q400_128WPS_2.createOrReplaceTempView('Q400_128WPS_2')

Q400_128WPS_2.createOrReplaceTempView('Q400_128WPS_2')

Q400_128WPS_2_1 = spark.sql('''SELECT MIN(`Time-Per-Sec`) AS TouchDownTime,
                
                flightdate,
                
                Filename
                
                FROM precogdb.q400128wps
                
                WHERE FlightDate > '2022-10-31'
                        
                AND `Radio-Altitude` NOT LIKE '%*%'
                        
                AND `Radio-Altitude` <> ''
                
                AND `Main-WOW` = 'Gnd (1)'
                
                AND `time-in-secs` > 2500
                        
                --AND  CAST(LoadedDate AS DATE) = CAST (current_timestamp AS DATE)

                GROUP BY flightdate,Filename ''')

Q400_128WPS_2_1.createOrReplaceTempView('Q400_128WPS_2_1')

Q400_128WPS_2_2 = spark.sql('''

            SELECT a.AirborneTime, b.TouchDownTime, a.Filename, a.Flightdate
            FROM Q400_128WPS_2 a 
            JOIN Q400_128WPS_2_1 b on a.Filename = b.Filename
            AND a.Flightdate = b.Flightdate
''')

Q400_128WPS_2_2.createOrReplaceTempView('Q400_128WPS_2_2')

Q400_128WPS_3 = spark.sql('''

                            SELECT filename, flightdate,
                            
                            CASE 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 1 THEN CONCAT('00000',CAST(`AirborneTime` as String)) 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 2 THEN CONCAT('0000',CAST(`AirborneTime` as String)) 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 3 THEN CONCAT('000',CAST(`AirborneTime` as String))
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 4 THEN CONCAT('00',CAST(`AirborneTime` as String)) 
                                WHEN LENGTH(CAST(`AirborneTime` as String)) = 5 THEN CONCAT('0',CAST(`AirborneTime` as String))
                                ELSE `AirborneTime`
                            END As AirborneTime,
                            CASE 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 1 THEN CONCAT('00000',CAST(`TouchdownTime` as String)) 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 2 THEN CONCAT('0000',CAST(`TouchdownTime` as String)) 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 3 THEN CONCAT('000',CAST(`TouchdownTime` as String))
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 4 THEN CONCAT('00',CAST(`TouchdownTime` as String)) 
                                WHEN LENGTH(CAST(`TouchdownTime` as String)) = 5 THEN CONCAT('0',CAST(`TouchdownTime` as String))
                                ELSE `TouchdownTime`
                            END As TouchdownTime
                            
                                                                
                            FROM Q400_128WPS_2_2
                                                
                    ''')

Q400_128WPS_3.createOrReplaceTempView('Q400_128WPS_3')

Q400_128WPS_4 = Q400_128WPS_3.withColumn("AirborneTime", regexp_replace(col("AirborneTime") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" )) \
                            .withColumn("TouchdownTime", regexp_replace(col("TouchdownTime") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

Q400_128WPS_4.createOrReplaceTempView('Q400_128WPS_4')

Q400_128WPS_4.show(truncate = False)

Q400_128WPS_5 = spark.sql('''

                SELECT to_timestamp(concat(flightdate,' ',AirborneTime)) AS Airborne_Time,
                
                to_timestamp(concat(flightdate,' ',TouchdownTime)) AS Touchdown_Time,
                
                Filename
                
                FROM Q400_128WPS_4


''')

Q400_128WPS_5.createOrReplaceTempView('Q400_128WPS_5')